## This program takes an input image of an parking lot and processes it to return the co-ordinates of the parking slot in a pickle file. The various config values for the image transforms are stored ia config file which is loaded into the program at the start. This program needs to be tuned at the start of the installation for a specific parking lot as a one time setup.

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import cv2
import os, glob
import numpy as np
from moviepy.editor import VideoFileClip
cwd = os.getcwd()
#importing Config file
import import_ipynb
import Config as c

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def show_images(images, cmap=None):
    cols = 2
    rows = (len(images)+1)//cols
    
    plt.figure(figsize=(15, 12))
    for i, image in enumerate(images):
        plt.subplot(rows, cols, i+1)
        # use gray scale color map if there is only one channel
        cmap = 'gray' if len(image.shape)==2 else cmap
        plt.imshow(image, cmap=cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [ ]:
# Load the Image
test_images = [plt.imread(path) for path in glob.glob(c.img_path)]
show_images(test_images)

### Color Selection and Edge Detection

In [ ]:
# image is expected be in RGB color space# image 
def select_rgb_white_yellow(image): 
    # white color mask
    lower = np.uint8([c.white_lower_r,c.white_lower_g,c.white_lower_b])
    upper = np.uint8([c.white_upper_r, c.white_upper_g, c.white_upper_b])
    white_mask = cv2.inRange(image, lower, upper)
    # yellow color mask
    lower = np.uint8([c.yellow_lower_r,c.yellow_lower_g,c.yellow_lower_b])
    upper = np.uint8([c.yellow_upper_r,c.yellow_upper_g,c.yellow_upper_b])
    yellow_mask = cv2.inRange(image, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

def convert_gray_scale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

white_yellow_images = list(map(select_rgb_white_yellow, test_images))
#show_images(white_yellow_images)

gray_images = list(map(convert_gray_scale, white_yellow_images))
show_images(gray_images)

### Gaussian Blur, Canny Transform and Hough line transform 

In [ ]:

def detect_edges(image, low_threshold=c.low_threshold, high_threshold=c.high_threshold):
    int_img = cv2.GaussianBlur(image, (3,15),0)
    #plt.imshow(int_img)
    return cv2.Canny(int_img, low_threshold, high_threshold)

def hough_lines(image):
    """
    `image` should be the output of a Canny transform.
    
    Returns hough lines (not the image with lines)
    """
    return cv2.HoughLinesP(image, rho=1, theta=np.pi/c.divisor, threshold=c.threshold_value, minLineLength=c.houghline_minlinelength, maxLineGap=c.houghline_maxlinegap)


def draw_lines(image, lines, color=[255, 0, 0], thickness=c.draw_lines_thickness, make_copy=True):
    # the lines returned by cv2.HoughLinesP has the shape (-1, 1, 4)
    if make_copy:
        image = np.copy(image) # don't want to modify the original
    cleaned = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y2-y1) <=c.lines_y_threshold_rect and abs(x2-x1) >=c.lines_x_threshold1_rect and abs(x2-x1) <= c.lines_x_threshold2_rect:
            #if abs(x2-x1) >=25 and abs(x2-x1) <= 50:
                cleaned.append((x1,y1,x2,y2))
                cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    print(" No lines detected: ", len(cleaned))
    return image

#Apply gaussian blur and canny transform 
edge_images = list(map(lambda image: detect_edges(image), gray_images))
#show_images(edge_images)
#show_images(edge_images)
# Hough line transform
list_of_lines = list(map(hough_lines, edge_images))

#Draw the lines on the image
line_images = []
for image, lines in zip(test_images, list_of_lines):
    line_images.append(draw_lines(image, lines))
    
show_images(line_images)

### Identify rectangular blocks of parking

In [ ]:
def identify_blocks(image, lines, make_copy=True):
    if make_copy:
        new_image = np.copy(image)
       
    #Step 1: Create a clean list of lines
    cleaned = []
    gap_list = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            if abs(y2-y1) <=c.lines_y_threshold and abs(x2-x1) >=c.lines_x_threshold1 and abs(x2-x1) <= c.lines_x_threshold2:
                cleaned.append((x1,y1,x2,y2))
                gap_list.append((y1))
    
    #Step 2: Sort cleaned by x1 position
    import operator
    list1 = sorted(cleaned, key=operator.itemgetter(0, 1))    
    #Step 3: Find clusters of x1 close together - clust_dist apart
    clusters = {}
    dIndex = 0   
    clus_dist = c.clust_dist

    for i in range(len(list1) - 1):
        distance = abs(list1[i+1][0] - list1[i][0])
        if distance <= clus_dist:
            if not dIndex in clusters.keys(): clusters[dIndex] = []
            clusters[dIndex].append(list1[i])
            clusters[dIndex].append(list1[i + 1])

        else:
            dIndex += 1
    
    #Step 4: Identify coordinates of rectangle around this cluster
    rects = {}
    i = 0
    for key in clusters:
        all_list = clusters[key]
        cleaned = list(set(all_list))
        if len(cleaned) > c.threshold_value:
            
            cleaned = sorted(cleaned, key=lambda tup: tup[1])
            avg_y1 = cleaned[0][1]
            avg_y2 = cleaned[-1][1]
            avg_x1 = c.max_image_width #put the max width of the image
            avg_x2 = 0
            for tup in cleaned:
                avg_x1 = min(avg_x1, tup[0])
                avg_x2 = max(avg_x2, tup[2])
            rects[i] = (avg_x1, avg_y1, avg_x2, avg_y2)
            i += 1
            
    print("Num Parking Lanes: ", len(rects))
    
    
    #Step 5: Draw the rectangles on the image
    buff = 0
    for key in rects:
        tup_topLeft = (int(rects[key][0] - buff), int(rects[key][1]))
        tup_botRight = (int(rects[key][2] + buff), int(rects[key][3]))
        cv2.rectangle(new_image, tup_topLeft,tup_botRight,(c.rect_color_r,c.rect_color_g,c.rect_color_b),c.rect_channels)
    return new_image, rects

# images showing the region of interest only
rect_images = []
rect_coords = []
for image, lines in zip(test_images, list_of_lines):
    new_image, rects = identify_blocks(image, lines)
    rect_images.append(new_image)
    rect_coords.append(rects)
    
show_images(rect_images)

### Identify each spot and count num of parking spaces

Next step- 
1. Based on width of each parking line segment into individual spots
2. draw a visualization of all parking spaces

In [ ]:
def draw_parking(image, rects, make_copy = True, color=[255, 0, 0], thickness=c.draw_lines_thickness, save = True):
    if make_copy:
        new_image = np.copy(image)
    gap = c.gap
    spot_dict = {} # maps each parking ID to its coords
    tot_spots = 0
    for key in rects:
        # Horizontal lines
        tup = rects[key]
        x1 = int(tup[0])
        x2 = int(tup[2])
        y1 = int(tup[1])
        y2 = int(tup[3])
        cv2.rectangle(new_image, (x1, y1),(x2,y2),(0,255,0),2)
        num_splits = int(abs(y2-y1)//gap)
        tot_spots += num_splits + 1
        #print(num_splits)
        for i in range(0, num_splits+1):
            y = int(y1 + i*gap)
            cv2.line(new_image, (x1, y), (x2, y), color, thickness)
            
        for i in range(0, num_splits+1):
            cur_len = len(spot_dict)
            y = int(y1 + i*gap)
            x = int(x1)
            spot_dict[(x1, y, x2, y+gap)] = cur_len +1
            
    
    print("total parking spaces: ", tot_spots, cur_len)
    if save:
        filename = 'test.jpg'
        cv2.imwrite(filename, new_image)
    return new_image, spot_dict

delineated = []
spot_pos = []
for image, rects in zip(test_images, rect_coords):
    new_image, spot_dict = draw_parking(image, rects)
    delineated.append(new_image)
    spot_pos.append(spot_dict) 
show_images(delineated)

In [ ]:
final_spot_dict = spot_pos[0]

In [ ]:
print(len(final_spot_dict))

In [ ]:
def assign_spots_map(image, spot_dict=final_spot_dict, make_copy = True, color=[255, 0, 0], thickness=c.draw_lines_thickness):
    if make_copy:
        new_image = np.copy(image)
    for spot in spot_dict.keys():
        (x1, y1, x2, y2) = spot
        cv2.rectangle(new_image, (int(x1),int(y1)), (int(x2),int(y2)), color, thickness)
    return new_image

marked_spot_images = list(map(assign_spots_map, test_images))
show_images(marked_spot_images)

In [ ]:
# Save spot dictionary as pickle file
import pickle
with open(c.pickle_file, 'wb') as handle:
    pickle.dump(final_spot_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)